In [1]:
import pandas as pd
import pypsa
import matplotlib.pyplot as plt

from numpy.testing import assert_allclose

In [6]:
status = pd.DataFrame(dtype=str)
status['climatic_data_source'] = ['PECD']
status['status'] = ['running']

status

,climatic_data_source,status
0,PECD,running


In [2]:
TY = 2030
CY = 1993
networks_dir = '../networks/'
resources_dir = '../../resources/'
short_names = {
    'Wind Onshore': 'onwind',
    'Wind Offshore': 'offwind',
    'Solar (Photovoltaic)': 'solar',
    'Run of River': 'ror',
    'Pondage': 'pondage',
    'Reservoir': 'hydro',
    'Pump Storage Open Loop': 'OLPHS',
    'Pump Storage Closed Loop': 'CLPHS',
}

In [3]:
def map_column_names(df, short_names):
    vre_gens = pd.DataFrame(index=df.columns)
    vre_gens['carrier'] = [c.split(' - ')[1] for c in df.columns]
    vre_gens['carrier_short'] = vre_gens['carrier'].map(short_names)
    vre_gens['node'] = [c.split(' - ')[0] for c in df.columns]
    vre_gens['name_short'] = vre_gens[['node', 'carrier_short']].agg(' '.join, axis=1)
    
    df.columns = df.columns.map(vre_gens['name_short'])

    return(df)

In [4]:
network_f = f'{networks_dir}/pilot_elec-vre_TY{TY}_{CY}.nc'
hydro_network_f = f'{networks_dir}/pilot_elec-vre-hydro_TY{TY}_{CY}.nc'
network = pypsa.Network(network_f)
hydro_network = pypsa.Network(hydro_network_f)

INFO:pypsa.io:Imported network pilot_elec-vre_TY2030_1993.nc has buses, carriers, generators, links, loads, stores
INFO:pypsa.io:Imported network pilot_elec-vre-hydro_TY2030_1993.nc has buses, carriers, generators, links, loads, stores


In [5]:
### check generators p_max_pu
p_max_pu_ref = pd.read_csv(f'{resources_dir}/climatic/PECD/TY{TY}/CY{CY}/generation_vre_timeseries.csv', header=0, index_col=0)
p_max_pu_ref = map_column_names(p_max_pu_ref, short_names)
dropgens = p_max_pu_ref.columns[~p_max_pu_ref.columns.isin(network.generators.index)] 
p_max_pu_ref.drop(dropgens, inplace=True, axis=1)

assert_allclose(network.generators_t.p_max_pu[p_max_pu_ref.columns], p_max_pu_ref)

In [6]:
### check ror inflow
gens_with_inflow = network.generators.index[network.generators.carrier.isin(['ror'])]
inflow_ref = pd.read_csv(f'{resources_dir}/climatic/PECD/TY{TY}/CY{CY}/hydro_inflow_timeseries.csv', header=0, index_col=0)
inflow_ref = map_column_names(inflow_ref, short_names)
inflow_p_max_pu_ref = (inflow_ref / network.generators.p_nom[gens_with_inflow]).dropna(axis=1).clip(upper=1.)

inflow_network = network.generators_t.p_max_pu.reindex(columns=gens_with_inflow, fill_value=1.)[inflow_p_max_pu_ref.columns]
assert_allclose(inflow_network, inflow_p_max_pu_ref)

In [7]:
### check hydro constraints
#### maximum generation
hydro_f = f'{resources_dir}/climatic/PECD/TY{TY}/hydro_uniform_Maximum Generated energy MWh per week.csv'
limits = pd.read_csv(hydro_f, header=0, index_col=0)
hydro_sus = pd.DataFrame(index=limits.columns)
hydro_sus['carrier'] = [c.split(' - ')[1] for c in limits.columns]
hydro_sus['carrier_short'] = hydro_sus['carrier'].map(short_names)
hydro_sus['node'] = [c.split(' - ')[0] for c in limits.columns]
hydro_sus['name_short'] = hydro_sus[['node', 'carrier_short']].agg(' '.join, axis=1)

limits.columns = limits.columns.map(hydro_sus['name_short'])
limits.index = network.snapshots
limits.drop(limits.columns[~limits.columns.isin(network.stores.index)], axis=1, inplace=True)

disp_links = network.links.index[network.links.bus0.isin(hydro_sus.name_short)]
max_disp_pu = limits / network.links.set_index(network.links.bus0).p_nom[limits.columns]
max_disp_pu.columns = [c+' dispatch' for c in max_disp_pu.columns]
limits.columns = [c+' dispatch' for c in limits.columns]
max_disp_pu0 = max_disp_pu.copy(deep=True)
# if link is already in p_max_pu, set p_max_pu to minimum of both constraints.
existing_p_max_pu = network.links_t.p_max_pu
max_disp_pu = max_disp_pu.where(
    max_disp_pu.reindex(columns=existing_p_max_pu.columns, fill_value=1.) < existing_p_max_pu, 
    existing_p_max_pu
).fillna(max_disp_pu)

In [8]:
disp_max = hydro_network.links_t.p_max_pu.reindex(columns=network.links.index, fill_value=1.).mul(network.links.p_nom)[limits.columns]
diff = limits - disp_max
diff.sum().round(2).sort_values()

SI00 CLPHS dispatch    0.0
SK00 hydro dispatch   -0.0
SK00 OLPHS dispatch    0.0
SK00 CLPHS dispatch   -0.0
NOS0 OLPHS dispatch    0.0
NON1 OLPHS dispatch    0.0
NOM1 OLPHS dispatch    0.0
dtype: float64

In [9]:
network.loads_t.p_set

Load,AL00 - Demand,AT00 - Demand,BA00 - Demand,BE00 - Demand,BG00 - Demand,CH00 - Demand,CZ00 - Demand,CY00 - Demand,DE00 - Demand,DKE1 - Demand,...,HU00 - EV,DE00 - EV,DKE1 - BAT,DKW1 - BAT,LT00 - BAT,MT00 - BAT,NL00 - BAT,SI00 - BAT,AT00 - BAT,DE00 - BAT
snapshot,,,,,,,,,,,,,,,,,,,,,
1993-01-01 00:00:00,841.883450,9549.374789,1513.158773,10550.225008,4508.694565,8738.404570,6568.832571,534.538255,80225.453692,2501.010398,...,262.646201,5949.290297,0.356987,0.531132,1.310418,0.059765,79.789504,0.248325,230.508370,0.000000
1993-01-01 01:00:00,737.800652,9293.595614,1425.855214,10011.675500,4427.597311,8959.256337,6564.914608,518.067100,79060.722114,2470.409466,...,216.397157,5554.582464,0.356987,4.444318,1.304153,0.058394,79.323539,0.243485,461.608216,0.000000
1993-01-01 02:00:00,681.680968,9154.278578,1378.419117,9612.277219,4328.850044,9109.653245,6466.868947,514.847669,78891.967884,2479.959535,...,112.089814,5404.930639,1.273548,9.367693,1.288818,0.057515,78.550800,0.241772,592.428118,0.000000
1993-01-01 03:00:00,662.571111,9003.271889,1349.994266,9370.757957,4128.444872,8649.496289,6360.768025,516.149036,78488.160650,2502.931800,...,46.443895,5510.184418,6.892294,0.531132,1.253242,0.057192,77.891947,0.240662,731.120310,0.000000
1993-01-01 04:00:00,662.494262,9035.570507,1346.297132,9499.510808,4056.888372,7265.623167,6131.727435,529.839014,77922.467322,2535.830585,...,33.389644,5957.463638,1.474531,0.531131,1.342005,0.057201,76.817255,0.241317,697.483504,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1993-12-31 19:00:00,1532.024698,11470.402585,1994.158178,12439.969518,5529.278414,5653.136466,8772.111891,695.443079,70923.790792,3051.927416,...,229.253729,9764.339046,-0.877988,-1.306289,1.859777,0.070783,97.816297,0.358978,-375.682521,-7.066975
1993-12-31 20:00:00,1478.090877,11189.475066,1926.661436,11854.063719,5282.726703,6287.448819,8343.226382,649.639361,68256.739436,2865.278771,...,162.272593,8743.418684,-0.723413,-1.076308,1.702238,0.069656,93.615784,0.335262,-204.012502,-7.066975
1993-12-31 21:00:00,1354.193622,10740.882109,1818.217998,11382.122372,4891.833075,7262.320797,7981.050472,596.974307,66488.373563,2663.154273,...,157.054320,8043.411962,-0.303126,-0.450997,1.585626,0.067197,89.736264,0.314240,-85.485805,-4.892521
